# Question Augmentation (similarity)

A frequent source of difficulty in using Translator Knowledge Graphs is the traversal of similar nodes.  If two non-identical (but very similar) concepts exist, they may be independently associated with other information by individual KPs.  If a user is unaware of this subtlety, it is easy to miss highly relevant answers.

Here is a simple query: (gene)--(chemical)--(disease).  This query might be used to explain gene/disease relationships that are driven by some metabolic process.   For instance, there is a known association between the gene SLC34A1 and Fanconi Syndrome:

In [48]:
import requests
import json
from notebook_functions import reasonerGraphToCytoscape, knowledgeGraphToCytoscape
from cyjupyter import Cytoscape
import time

In [49]:
class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        return r.json()
        
    
strider = Strider()

## The relationship between the gene and disease is a known one.

In [50]:
question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:6569'},
                      {'id': 'n1', 'type':'disease', 'curie': 'MONDO:0001083'}],
             'edges': [ {'id': 'e0', 'source_id': 'n0', 'target_id': 'n1', 'type': 'contributes_to'}]}
message = {'message': {'query_graph': question}}

In [51]:
c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'gene\n[NCBIGene:6569]', 'curie': 'NCBIG…

In [52]:
q = strider.send_message(message)
print(q)

a4d6c18d-e3b3-491e-b907-c51e7752aa31


In [54]:
r=strider.query_result(q)
while len(r['results']) == 0:
    time.sleep(1)
    print('checking')
    r = strider.query_result(q)
print('results found')

results found


In [55]:
c=knowledgeGraphToCytoscape(r['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'NCBIGene:6569', 'label': 'SLC34A1\n[NCBIGene:6569]', '…

## There is no chemical relating the gene to the disease

In [56]:
question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:6569'},
                       {'id':'n1', 'type':'chemical_substance'},
                      {'id': 'n2', 'type':'disease', 'curie':'MONDO:0001083'}],
             'edges': [ {'id': 'e0', 'source_id': 'n0', 'target_id': 'n1', 'type':'increases_degradation_of'},
                        {'id': 'e1', 'source_id': 'n1', 'target_id': 'n2', 'type':'related_to'}]}
message = {'message': {'query_graph': question}}

c=reasonerGraphToCytoscape(question)
c['style'][0]['style']['width']='10em'
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})


Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'gene\n[NCBIGene:6569]', 'curie': 'NCBIG…

In [57]:
q = strider.send_message(message)

In [60]:
r=strider.query_result(q)
print(r['results'])

[]


## We can augment the question with similarity

In [61]:
qa_url = 'https://questionaugmentation.renci.org/node_expand'
rq = requests.post(qa_url,json=message)
new_questions = rq.json()
newq = new_questions[0]

c=reasonerGraphToCytoscape(newq['query_graph'])
c['style'][0]['style']['width']='12em'
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={ "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'gene\n[NCBIGene:6569]', 'curie': 'NCBIG…

In [63]:
newmessage = {'message': newq}
q = strider.send_message(newmessage)

In [66]:
r=strider.query_result(q)
while len(r['results']) == 0:
    time.sleep(1)
    print('checking')
    r = strider.query_result(q)
print('results found')

results found


In [67]:
c=knowledgeGraphToCytoscape(r['knowledge_graph'])
Cytoscape(data=c, visual_style=c["style"], layout_name='cose', layout={"height": "400px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'NCBIGene:6569', 'label': 'SLC34A1\n[NCBIGene:6569]', '…